# 这里我们以乡村和城镇的个人最高学历为例进行数据可视化

### 一、提取数据

#### 1.各个最高学历人数
    在数据库文档中可以查询到个人库中的w01变量代表的是个人最高学历

In [18]:
import pymysql

edu_mean = {
    0:"文盲/半文盲",
    10:"没有上过学",
    3:"小学",
    4:"初中",
    5:"高中/中专/技校/职高",
    6:"大专",
    7:"大学本科",
    8:"硕士",
    9:"博士"
}
edu_list = list(edu_mean.values());

#在这里输入字段名和字段所在表名
field = "w01"
table = "perso"

#读数据库数据
conn = pymysql.connect(host="localhost", port=3306, passwd="1234", user="root", db="empt")#连接到数据库
cursor = conn.cursor()#获取游标

#读取人数
count_list = []
for i in edu_mean.keys():
    cmd = "SELECT COUNT(%s) FROM %s WHERE %s=%d;"%(field, table, field, i)
    cursor.execute(cmd)
    data = cursor.fetchall()
    for j in data:
        count_list.append(int(j[0]))
count_list

[7104, 756, 3598, 5273, 3058, 1648, 1335, 112, 7]

#### 2.乡村城镇各个最高学历人数
    同样的，变量urban18代表乡镇类型

In [19]:
urban_mean = {
    0:"乡村",
    1:"城镇"
}

#读取城乡各个学历人数
con_count_list = []
urb_count_list = []
for i in edu_mean.keys():
    cmd = "SELECT COUNT(%s) FROM %s WHERE %s=%d AND urban18=0;"%(field, table, field,i)
    cursor.execute(cmd)
    data = cursor.fetchall()
    for j in data:
        con_count_list.append(int(j[0]))
for i in edu_mean.keys():
    cmd = "SELECT COUNT(%s) FROM %s WHERE %s=%d AND urban18=1;"%(field, table, field, i)
    cursor.execute(cmd)
    data = cursor.fetchall()
    for j in data:
        urb_count_list.append(int(j[0]))

print(con_count_list)
print(urb_count_list)

[3164, 566, 2200, 2672, 1094, 424, 237, 15, 0]
[2045, 189, 1367, 2505, 1908, 1172, 1054, 91, 7]


### 二、数据可视化

#### 1.单数据系列柱状图

In [51]:
import pymysql
from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.commons.utils import JsCode
from pyecharts.globals import ThemeType
from pyecharts.options import TitleOpts

x_data = edu_list
y_data = count_list

bar = (Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
       .add_xaxis(x_data)
       .add_yaxis("人数",y_data,category_gap="50%")
       .set_global_opts(title_opts=opts.TitleOpts(title="1.单数据系列柱状图"))
)
bar.render("./bar1.html")
bar.render_notebook()


#### 2.多数据系列柱状图

In [52]:
x_data = edu_list
y_data1 = con_count_list
y_data2 = urb_count_list

bar = (Bar()
       .add_xaxis(x_data)
       .add_yaxis("乡村人数",y_data1)
       .add_yaxis("城镇人数",y_data2)  
       .set_global_opts(title_opts=opts.TitleOpts(title="2.多数据系列柱状图"))
)
bar.render("./bar2.html")
bar.render_notebook()


#### 3.堆积柱形图

In [49]:
x_data = edu_list
y_data1 = con_count_list
y_data2 = urb_count_list

bar = (
        Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
       .add_xaxis(x_data)
       .add_yaxis("乡村人数",y_data1,stack="stack1",category_gap="50%")
       .add_yaxis("城镇人数",y_data2,stack="stack1",category_gap="50%") 
       .set_global_opts(title_opts=opts.TitleOpts(title="3.堆积柱形图"))
       .set_series_opts(
            label_opts=opts.LabelOpts(
            position="right"
            )
        )
)
bar.render("./bar3.html")
bar.render_notebook()


'/home/admin/bar3.html'

#### 4.饼图

In [53]:
from pyecharts.charts import Pie

x_data = edu_list
y_data = count_list

pie = (
    Pie(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
    .add("", [list(z) for z in zip(x_data, y_data)])
    .set_global_opts(title_opts=opts.TitleOpts(title="4.饼图"))
    

)
pie.render("./pie1.html")
pie.render_notebook()

#### 5.圆环图

In [24]:
x_data = edu_list
y_data = count_list

pie = (
    Pie(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
    .add("", [list(z) for z in zip(x_data, y_data)],radius=["30%", "55%"],)
    .set_global_opts(title_opts=opts.TitleOpts(title="5.圆环图"),
                    legend_opts=opts.LegendOpts(type_="scroll", pos_left="75%", orient="vertical"))
)
pie.render("./pie2.html")
pie.render_notebook()

#### 6.单数据南丁格尔玫瑰图

In [54]:
x_data = edu_list
y_data = count_list

pie = (
    Pie(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
    .add("", [list(z) for z in zip(x_data, y_data)],radius=["10%", "55%"], rosetype="radius")
    .set_global_opts(title_opts=opts.TitleOpts(title="6.单数据南丁格尔玫瑰图"),
                    legend_opts=opts.LegendOpts(type_="scroll", pos_left="75%", orient="vertical"))
    

)
pie.render("./pie3.html")
pie.render_notebook()

#### 7.多数据极坐标图

In [55]:
from pyecharts.charts import Polar

x_data = edu_list
y_data1 = con_count_list
y_data2 = urb_count_list

polar = (
    Polar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
    .add_schema(angleaxis_opts=opts.AngleAxisOpts(data=x_data, type_="category"))
    .add("乡村", y_data1, type_ = "bar", stack = "stack1")
    .add("城镇", y_data2, type_ = "bar", stack = "stack1")
    .set_global_opts(title_opts =opts.TitleOpts(title="7.多数据南丁格尔玫瑰图"))
)
polar.render("./polar1.html")
polar.render_notebook()

#### 8.极坐标图

In [56]:
x_data = edu_list
y_data1 = con_count_list
y_data2 = urb_count_list

polar = (
    Polar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
    .add_schema(
        radiusaxis_opts=opts.RadiusAxisOpts(data=x_data, type_="category"),
        angleaxis_opts=opts.AngleAxisOpts(is_clockwise=True, max_=6000),
    )
    .add("乡村", y_data1, type_ = "bar", stack = "stack1")
    .add("城镇", y_data2, type_ = "bar", stack = "stack1")
    .set_global_opts(title_opts =opts.TitleOpts(title="8.极坐标图"))
)
polar.render("./polar2.html")
polar.render_notebook()

#### 9.单数据雷达图

In [28]:
from pyecharts.charts import Radar

x_data = edu_list
y_data = count_list

radar = (
    Radar()
    .add_schema(
        schema=[
            opts.RadarIndicatorItem(name=x_data[0], max_=7500),
            opts.RadarIndicatorItem(name=x_data[1], max_=7500),
            opts.RadarIndicatorItem(name=x_data[2], max_=7500),
            opts.RadarIndicatorItem(name=x_data[3], max_=7500),
            opts.RadarIndicatorItem(name=x_data[4], max_=7500),
            opts.RadarIndicatorItem(name=x_data[5], max_=7500),
            opts.RadarIndicatorItem(name=x_data[6], max_=7500),
            opts.RadarIndicatorItem(name=x_data[7], max_=7500),
            opts.RadarIndicatorItem(name=x_data[8], max_=7500),
        ])
    .add("人数",[y_data])
    .set_global_opts(title_opts =opts.TitleOpts(title="9.单数据雷达图"))
)
radar.render("./radar1.html")
radar.render_notebook()


#### 10.多数据雷达图

In [57]:
x_data = edu_list
y_data1 = con_count_list
y_data2 = urb_count_list

radar = (
    Radar()
    .add_schema(
        schema=[
            opts.RadarIndicatorItem(name=x_data[0], max_=3500),
            opts.RadarIndicatorItem(name=x_data[1], max_=3500),
            opts.RadarIndicatorItem(name=x_data[2], max_=3500),
            opts.RadarIndicatorItem(name=x_data[3], max_=3500),
            opts.RadarIndicatorItem(name=x_data[4], max_=3500),
            opts.RadarIndicatorItem(name=x_data[5], max_=3500),
            opts.RadarIndicatorItem(name=x_data[6], max_=3500),
            opts.RadarIndicatorItem(name=x_data[7], max_=3500),
            opts.RadarIndicatorItem(name=x_data[8], max_=3500),
        ],shape="circle")
    .add("乡村",[y_data1],color="#f9713c")
    .add("城镇",[y_data2],color="#b3e4a1")
    .set_global_opts(title_opts =opts.TitleOpts(title="10.多数据雷达图"))
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
)
radar.render("./radar2.html")
radar.render_notebook()


#### 11.散点图

In [58]:
from pyecharts.charts import Scatter

x_data = edu_list
y_data = count_list

scatter = (
    Scatter(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
    .add_xaxis(x_data)
    .add_yaxis("人数",y_data)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="11.散点图"),
        xaxis_opts=opts.AxisOpts(
        axistick_opts=opts.AxisTickOpts(is_align_with_label=True),
        is_scale=False,
        name_rotate=60,
        axislabel_opts={"rotate":45})
    )
)
scatter.render("./scatter1.html")
scatter.render_notebook()

#### 12.折线图

In [59]:
import pyecharts.options as opts
from pyecharts.charts import Line

x_data = edu_list
y_data = count_list

line = Line()
line.add_xaxis(xaxis_data = x_data)
line.add_yaxis(
    series_name="人数",
    y_axis=y_data,
    label_opts=opts.LabelOpts(is_show=False),
)
line.set_global_opts(
    title_opts=opts.TitleOpts(title="12.折线图"),
    xaxis_opts=opts.AxisOpts(
        axistick_opts=opts.AxisTickOpts(is_align_with_label=True),
        is_scale=False,
        name_rotate=60,
        axislabel_opts={"rotate":45})
    )
line.render("./line1.html")
line.render_notebook()


#### 13.折线堆叠图

In [60]:
import pyecharts.options as opts
from pyecharts.charts import Line

x_data = edu_list
y_data1 = con_count_list
y_data2 = urb_count_list

line = Line()
line.add_xaxis(xaxis_data = x_data)
line.add_yaxis(
    series_name="乡村",
    stack = "stack1",
    y_axis=y_data1,
    label_opts=opts.LabelOpts(is_show=False),
)
line.add_yaxis(
    series_name="城市",
    stack = "stack1",
    y_axis=y_data2,
    label_opts=opts.LabelOpts(is_show=False),
)
line.set_global_opts(
    title_opts=opts.TitleOpts(title="13.折线堆叠图"),
    xaxis_opts=opts.AxisOpts(
        axistick_opts=opts.AxisTickOpts(is_align_with_label=True),
        is_scale=False,
        name_rotate=60,
        axislabel_opts={"rotate":45})
    )
line.render("./line2.html")
line.render_notebook()

#### 14.折线柱状堆叠图

In [61]:
from pyecharts import options as opts
from pyecharts.charts import Bar, Grid, Line

x_data = edu_list
y_line = urb_count_list
y_bar = count_list

bar = (
    Bar()
    .add_xaxis(x_data)
    .add_yaxis(
        "总人数",
        y_bar,
        yaxis_index=0,
        color="#aaaaaa",        
    )
    .set_global_opts(
        yaxis_opts=opts.AxisOpts(
            name="总人数",
            position="left",
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color="#5793f3")
            ),
        ),
        title_opts=opts.TitleOpts(title="14.折线柱状堆叠图"),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
        xaxis_opts=opts.AxisOpts(
        axistick_opts=opts.AxisTickOpts(is_align_with_label=True),
        is_scale=False,
        name_rotate=60,
        axislabel_opts={"rotate":45})
    )
)


line = (
    Line()
    .add_xaxis(x_data)
    .add_yaxis(
        "城镇人数",
        y_line,
        yaxis_index=0,    #删了一个Y轴，Y轴索引由2改为1
        color="#675bff",
        label_opts=opts.LabelOpts(is_show=False),
        z = 20
    )
)

bar.overlap(line)
grid = Grid()
grid.add(bar, opts.GridOpts(pos_left="5%", pos_right="20%"), is_control_axis_index=True)
grid.render("./bar_line.html")
grid.render_notebook()

#### 15.折线面积图

In [62]:
import pyecharts.options as opts
from pyecharts.charts import Line

x_data = edu_list
y_data = count_list

line = Line()
line.add_xaxis(xaxis_data = x_data)
line.add_yaxis(
    series_name="人数",
    y_axis=y_data,
    label_opts=opts.LabelOpts(is_show=False),
)
line.set_series_opts(
    areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
    label_opts=opts.LabelOpts(is_show=False),
)
line.set_global_opts(
    title_opts=opts.TitleOpts(title="15.折线面积图"),
    xaxis_opts=opts.AxisOpts(
        axistick_opts=opts.AxisTickOpts(is_align_with_label=True),
        is_scale=False,
        boundary_gap=False,
        name_rotate=60,
        axislabel_opts={"rotate":45})
)
line.render("./line3.html")
line.render_notebook()

#### 16.双维度曲线面积图 ####

In [63]:
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.faker import Faker

x_data = edu_list
y_data1 = urb_count_list
y_data2 = con_count_list

line = Line()
line.add_xaxis(x_data)
line.add_yaxis("城镇", y_data1, is_smooth=True)
line.add_yaxis("乡村", y_data2, is_smooth=True)
line.set_series_opts(
    areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
    label_opts=opts.LabelOpts(is_show=False),
)
line.set_global_opts(
    title_opts=opts.TitleOpts(title="16.双维度曲线面积图"),
    xaxis_opts=opts.AxisOpts(
        axistick_opts=opts.AxisTickOpts(is_align_with_label=True),
        is_scale=False,
        boundary_gap=False,
        name_rotate=60,
        axislabel_opts={"rotate":45})
)
line.render("./line4.html")
line.render_notebook()


#### 17.平行关系图

In [64]:
from pyecharts.charts import Parallel

x_data = edu_list
y_data1 = urb_count_list
y_data2 = con_count_list

parallel_axis = [{"dim": 0, "name": x_data[0],"min":0,"max":3500},
    {"dim": 2, "name": x_data[2],"min":0,"max":3500},
    {"dim": 3, "name": x_data[3],"min":0,"max":3500},
    {"dim": 4, "name": x_data[4],"min":0,"max":3500},
    {"dim": 5, "name": x_data[5],"min":0,"max":3500},
    {"dim": 6, "name": x_data[6],"min":0,"max":3500},
    {"dim": 7, "name": x_data[7],"min":0,"max":3500},
    {"dim": 8, "name": x_data[8],"min":0,"max":3500}
                ]

parallel = (Parallel()
    .add_schema(schema=parallel_axis)
    .add("乡村",data=[y_data1])
    .add("城镇",data=[y_data2])
    .set_global_opts(title_opts =opts.TitleOpts(title="17.平行关系图"))
)

parallel.render("./parallel1.html")
parallel.render_notebook()

#### 18.漏斗图

In [65]:
from pyecharts.charts import Funnel

x_data = edu_list
y_data = count_list

funnel = (Funnel()
          .add("人数",[list(z) for z in zip(x_data, y_data)])
          .set_global_opts(title_opts=opts.TitleOpts(title="18.漏斗图",pos_top="15%"))
)
funnel.render("./funnel.html")
funnel.render_notebook()

#### 19.阶梯折线图

In [66]:
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.faker import Faker

x_data = edu_list
y_data = count_list

c = Line()
c.add_xaxis(x_data)
c.add_yaxis("总人数", y_data, is_step=True)
c.set_global_opts(title_opts=opts.TitleOpts(title="居民受教育程度"),
                 xaxis_opts=opts.AxisOpts(
        axistick_opts=opts.AxisTickOpts(is_align_with_label=True),
        is_scale=False,
        name_rotate=60,
        axislabel_opts={"rotate":45}))

c.render("./line_step1.html")
c.render_notebook()



#### 20.多维度折线图

In [67]:
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.faker import Faker

x_data = edu_list
y_data1 = urb_count_list
y_data2 = con_count_list

c = Line()
c.add_xaxis(x_data)
c.add_yaxis("城镇人数", y_data1, is_step=True)
c.add_yaxis("乡村人数", y_data2, is_step=True)
c.set_global_opts(title_opts=opts.TitleOpts(title="城乡居民受教育程度"),
                 xaxis_opts=opts.AxisOpts(
        axistick_opts=opts.AxisTickOpts(is_align_with_label=True),
        is_scale=False,
        name_rotate=60,
        axislabel_opts={"rotate":45}))

c.render("./line_step2.html")
c.render_notebook()

#### 21.双饼图

In [68]:
x_data = edu_list
y_data1 = urb_count_list
y_data2 = con_count_list

pieBir = Pie(init_opts=opts.InitOpts(width='700px',height='300px'))
pieBir.add(series_name='城镇人数',data_pair = [list(z) for z in zip(x_data,y_data1)], center=[200, 300],radius=[70,140]) 
pieBir.set_global_opts(title_opts=opts.TitleOpts(title="城镇受教育情况",pos_left="15%",pos_bottom="2%"),   )

pieBirE = Pie(init_opts=opts.InitOpts(width='700px',height='300px'))
pieBirE.add(series_name='乡村人数',data_pair = [list(z) for z in zip(x_data,y_data2)],center=[650, 300],radius=[70,140])   #饼图圆心位置
pieBirE.set_global_opts(title_opts=opts.TitleOpts(title="乡村受教育情况",pos_right='20%',pos_bottom="2%"),    #标题位置
                    )  
  
grid = Grid()
grid.add(pieBir, grid_opts=opts.GridOpts(pos_right="5%"))
grid.add(pieBirE, grid_opts=opts.GridOpts(pos_left="95%"))
grid.render("./double_pie.html")
grid.render_notebook()